In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder

In [2]:
AwanTunai_data = pd.read_excel('20220413_Northeastern_AwanTunai_Capstone_Data.xlsx', keep_default_na = 'TRUE')

In [3]:
AwanTunai_data.head()

,id,order_id,merchant_id,sku_id,date,top_cat_id,sub_cat_id,price
0,654713,82186,1754,981,2021-06-29,7.0,44.0,877000.0
1,654712,82186,1754,23060,2021-06-29,7.0,44.0,877000.0
2,654711,82186,1754,1548,2021-06-29,7.0,44.0,877000.0
3,654714,82186,1754,1969,2021-06-29,7.0,44.0,877000.0
4,654687,82186,1754,343,2021-06-29,7.0,44.0,312000.0


In [4]:
AwanTunai_data.isna().sum()

id                0
order_id          0
merchant_id       0
sku_id            0
date              0
top_cat_id     2208
sub_cat_id     2208
price             0
dtype: int64

In [5]:
na_AwanTunai_data = AwanTunai_data[AwanTunai_data["top_cat_id"].isna()]
AwanTunai_data = AwanTunai_data.dropna(subset=["top_cat_id"])

In [6]:
AwanTunai_data.isna().sum()

id             0
order_id       0
merchant_id    0
sku_id         0
date           0
top_cat_id     0
sub_cat_id     0
price          0
dtype: int64

In [7]:
print(AwanTunai_data.dtypes) 

id                      int64
order_id                int64
merchant_id             int64
sku_id                  int64
date           datetime64[ns]
top_cat_id            float64
sub_cat_id            float64
price                 float64
dtype: object


In [8]:
AwanTunai_sku_movement = pd.read_csv('sku_movement.csv')

In [9]:
AwanTunai_sku_movement.head()

,sku_id,Avg_diff
0,0,7.812500
1,1,4.750000
2,2,4.068966
3,3,3.051282
4,4,3.169014


In [10]:
AwanTunai_data["sku_movement"] = 0

In [11]:
AwanTunai_data.sku_movement.update(AwanTunai_data.sku_id.map(AwanTunai_sku_movement.set_index('sku_id').Avg_diff))

In [12]:
AwanTunai_data = AwanTunai_data[AwanTunai_data['sku_movement'] > 0]

In [13]:
AwanTunai_data['sku_movement'] = AwanTunai_data['sku_movement'].round(decimals = 2)

In [14]:
AwanTunai_data.head()

,id,order_id,merchant_id,sku_id,date,top_cat_id,sub_cat_id,price,sku_movement
0,654713,82186,1754,981,2021-06-29,7.0,44.0,877000.0,42.33
1,654712,82186,1754,23060,2021-06-29,7.0,44.0,877000.0,52.67
2,654711,82186,1754,1548,2021-06-29,7.0,44.0,877000.0,22.67
4,654687,82186,1754,343,2021-06-29,7.0,44.0,312000.0,17.80
5,1058235,129785,1760,1548,2021-10-04,7.0,44.0,3750.0,22.67


In [15]:
bins = [1, 15, 30, 45, (AwanTunai_sku_movement["Avg_diff"]).max()]
df = pd.cut(AwanTunai_sku_movement["Avg_diff"], bins)
df.value_counts()

(1.0, 15.0]     409
(15.0, 30.0]    126
(30.0, 45.0]     32
(45.0, 64.0]     12
Name: Avg_diff, dtype: int64

In [16]:
df_merchant_sku_N_movement = AwanTunai_data[['merchant_id', 'sku_movement', 'sku_id']]
df_merchant_sku_N_movement = df_merchant_sku_N_movement.drop_duplicates(subset = ['merchant_id', 'sku_id'])

In [17]:
df_merchant_sku_N_movement.head()

,merchant_id,sku_movement,sku_id
0,1754,42.33,981
1,1754,52.67,23060
2,1754,22.67,1548
4,1754,17.80,343
5,1760,22.67,1548


In [18]:
from scipy.sparse import csr_matrix
from fuzzywuzzy import fuzz
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

C:\Users\Uzzi\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [19]:
df_movement_day_diff_features = df_merchant_sku_N_movement.pivot(
    index = 'sku_id',
    columns = 'merchant_id',
    values = 'sku_movement'
).fillna(0)

In [20]:
df_movement_day_diff_features

merchant_id,875,1016,1041,1103,1156,1160,1161,1170,1171,1172,...,2885,2891,2913,2932,2953,2954,2973,3075,3104,3114
sku_id,,,,,,,,,,,,,,,,,,,,,
0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,7.81,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,0.0,0.0,0.0,4.75,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00,0.0,0.0,0.0,4.07,4.07,0.0,0.00,0.0,4.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.0,0.0,0.0,3.05,3.05,0.0,0.00,0.0,3.05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.17,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23746,0.00,0.0,0.0,0.0,0.00,0.00,0.0,2.78,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23747,0.00,0.0,0.0,0.0,0.00,0.00,0.0,8.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23748,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,7.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
sku_list = pd.DataFrame(df_merchant_sku_N_movement.sku_id.unique())
sku_list.columns = ['sku_id']
sku_list = sku_list.sort_values(by='sku_id', ascending=True).reset_index()
sku_list.insert(loc=0, column='row_num', value=np.arange(len(sku_list)))
sku_list = sku_list[['row_num', 'sku_id']]
sku_list.head()

,row_num,sku_id
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4


In [22]:
sku_movement_to_idx = sku_list.to_dict()
sku_movement_to_idx = sku_movement_to_idx['sku_id']

In [23]:
sku_movement_to_idx

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 8,
 6: 9,
 7: 15,
 8: 18,
 9: 21,
 10: 22,
 11: 25,
 12: 27,
 13: 33,
 14: 34,
 15: 38,
 16: 40,
 17: 45,
 18: 56,
 19: 60,
 20: 62,
 21: 65,
 22: 76,
 23: 77,
 24: 83,
 25: 85,
 26: 88,
 27: 89,
 28: 96,
 29: 103,
 30: 105,
 31: 106,
 32: 115,
 33: 122,
 34: 139,
 35: 143,
 36: 144,
 37: 145,
 38: 156,
 39: 158,
 40: 159,
 41: 160,
 42: 165,
 43: 166,
 44: 177,
 45: 190,
 46: 210,
 47: 214,
 48: 218,
 49: 221,
 50: 226,
 51: 232,
 52: 253,
 53: 257,
 54: 272,
 55: 274,
 56: 283,
 57: 289,
 58: 306,
 59: 313,
 60: 317,
 61: 343,
 62: 354,
 63: 360,
 64: 371,
 65: 383,
 66: 385,
 67: 387,
 68: 400,
 69: 401,
 70: 404,
 71: 412,
 72: 414,
 73: 415,
 74: 419,
 75: 436,
 76: 437,
 77: 444,
 78: 448,
 79: 466,
 80: 468,
 81: 473,
 82: 477,
 83: 482,
 84: 503,
 85: 519,
 86: 523,
 87: 526,
 88: 540,
 89: 547,
 90: 551,
 91: 562,
 92: 567,
 93: 568,
 94: 592,
 95: 600,
 96: 609,
 97: 621,
 98: 628,
 99: 643,
 100: 655,
 101: 667,
 102: 678,
 103: 688,
 1

In [24]:
matrix_movement_day_diff_features = csr_matrix(df_movement_day_diff_features.values)

In [25]:
def fuzzy_matching(mapper, selected_sku, verbose=True):
    match_tuple = []
    # get match
    for idx, sku_id in mapper.items():
        ratio = fuzz.ratio(str(sku_id), str(selected_sku))
        if ratio == 100:
            match_tuple.append((sku_id, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key = lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        sku_movement = AwanTunai_sku_movement[AwanTunai_sku_movement["sku_id"] == 20315]["Avg_diff"]\
        .values[0].round(decimals = 2)
        is_fast = "Fast Moving"
        if sku_movement > 10:
            is_fast = "Slow Moving"
        print('Found possible matches in our database: {0} [{1}]'.format([x[0] for x in match_tuple], is_fast))
        print('{0} moving with a speed of {1} days on an average.\n'.format(selected_sku, sku_movement))
    return match_tuple[0][1]

In [26]:
def make_recommendation(model_knn, data, mapper, selected_sku, n_recommendations):
    # fit
    model_knn.fit(data)
    # get input movie index
    print('You have input SKU:', selected_sku)
    idx = fuzzy_matching(mapper, selected_sku, verbose = True)
    if(idx is not None):
        # inference
        print('Recommendation system start to make inference')
        print('......\n')
        distances, indices = model_knn.kneighbors(data[idx], n_neighbors = n_recommendations)
        # get list of raw idx of recommendations
        raw_recommends = \
            sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
        # print recommendations
        print('Recommendations for {}:'.format(selected_sku))
        for i, (idx, dist) in enumerate(raw_recommends):
            try:
                print('{0}: {1}, with distance of {2}'.format(i+1, mapper[idx], dist))
            except:
                print("{0}: Exception thrown. {1} SKU does not exist.".format(i+1, idx))

In [27]:
# define model
model_knn = NearestNeighbors(metric='cosine',\
                             algorithm='brute',\
                             n_neighbors = 3,\
                             n_jobs = -1)

In [28]:
selected_sku = '20315'

make_recommendation(
    model_knn = model_knn,
    selected_sku = selected_sku,
    data = matrix_movement_day_diff_features,
    mapper = sku_movement_to_idx,
    n_recommendations = 11)

You have input SKU: 20315
Found possible matches in our database: [20315] [Fast Moving]
20315 moving with a speed of 6.07 days on an average.

Recommendation system start to make inference
......

Recommendations for 20315:
1: 306, with distance of 0.7962152135151943
2: 20889, with distance of 0.7817821097640076
3: 16280, with distance of 0.7776252050016697
4: 7209, with distance of 0.7609542781331213
5: 19477, with distance of 0.758253110792386
6: 20313, with distance of 0.746453723581445
7: 23185, with distance of 0.746453723581445
8: 19281, with distance of 0.7327387580875757
9: 16640, with distance of 0.6913933000758162
10: 1865, with distance of 0.6913933000758162
